# GYM Taxi environment with Q-learning

**Description:** Simple Q-learning using taxi environment GYM time<br>
There are 4 locations objective is to pick passenger at one location and drop him off in another.<br>
Penalties: -1 for each timestep, -10 for illegal dropoff/pickup<br>
Bonus: +20 for each dropoff<br>
https://gym.openai.com/envs/Taxi-v2/


In [1]:
import gym
from IPython.display import clear_output
from time import sleep
import numpy as np
import random

In [2]:
# supporting functions
def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'])
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        sleep(.1)

In [3]:
env = gym.make("Taxi-v3").env
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [4]:
env.reset() # reset environment to a new, random state
env.render()

print("Action Space {}".format(env.action_space))
print("State Space {}".format(env.observation_space))

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

Action Space Discrete(6)
State Space Discrete(500)


#### Solve enviroment without reinforcement
(using a q-table)

In [5]:
env.s = 328  # set environment to illustration's state

epochs = 0
penalties, reward = 0, 0

frames = [] # for animation

done = False

while not done:
    action = env.action_space.sample()
    state, reward, done, info = env.step(action)

    if reward == -10:
        penalties += 1
    
    # Put each rendered frame into dict for animation
    frames.append({
        'frame': env.render(mode='ansi'),
        'state': state,
        'action': action,
        'reward': reward
        }
    )

    epochs += 1
       
print("Timesteps taken (epochs): {}".format(epochs))
print("Penalties incurred: {}".format(penalties))

Timesteps taken (epochs): 2761
Penalties incurred: 883


In [6]:
print_frames(frames)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 2761
State: 0
Action: 5
Reward: 20


#### Using Q-learning to train via reinforcement 
(using Q-learning)

In [7]:
q_table = np.zeros([env.observation_space.n, env.action_space.n])

In [8]:
%%time
# Training the agent

# Hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.1

# For plotting metrics
all_epochs = []
all_penalties = []

for i in range(1, 100001):
    state = env.reset()

    epochs, penalties, reward, = 0, 0, 0
    done = False
    
    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # Explore action space
        else:
            action = np.argmax(q_table[state]) # Exploit learned values

        next_state, reward, done, info = env.step(action) 
        
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value

        if reward == -10:
            penalties += 1

        state = next_state
        epochs += 1
        
    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episode: {i}")

print("Training finished.\n")

Episode: 100000
Training finished.

CPU times: user 48.9 s, sys: 12.3 s, total: 1min 1s
Wall time: 46 s


#### Evaluate agent

In [9]:
# Evaluate agent's performance after Q-learning
frames = []

total_epochs, total_penalties = 0, 0
episodes = 100

for _ in range(episodes):
    state = env.reset()
    epochs, penalties, reward = 0, 0, 0
    
    done = False
    
    while not done:
        action = np.argmax(q_table[state])
        state, reward, done, info = env.step(action)
        # Put each rendered frame into dict for animation
        frames.append({
           'frame': env.render(mode='ansi'),
           'state': state,
           'action': action,
           'reward': reward
        }
        )

        if reward == -10:
            penalties += 1

        epochs += 1

    total_penalties += penalties
    total_epochs += epochs

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")

Results after 100 episodes:
Average timesteps per episode: 13.39
Average penalties per episode: 0.0


In [ ]:
print_frames(frames)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)

Timestep: 635
State: 372
Action: 0
Reward: -1
